In [1]:
import geopandas as gpd
import rasterio as rio
import os
from rasterio.windows import from_bounds
#from rasterio.enums import Resampling

In [2]:
os.environ["AWS_REQUEST_PAYER"] = "requester"

In [3]:
def clip_by_geojson(infile,outfile,geojson_file):
    
    filepath=infile
    geoms = gpd.read_file(geojson_file)
    print(geoms)
    left = geoms.bounds.minx[0]
    right = geoms.bounds.maxx[0]
    top = geoms.bounds.maxy[0]
    bottom = geoms.bounds.miny[0]
    
    print(left,right,top,bottom)

    with rio.open(filepath) as src:
        my_window = from_bounds(left, bottom, right, top, src.transform)
        print(my_window)

        rst = src.read(1, window=my_window)
        
        py, px = src.index(left, top)
        print('Pixel Y, X coords: {}, {}'.format(py, px))
        
        py, px = src.index(-90, 49.98)
        print('Pixel Y, X coords: {}, {}'.format(py, px))
        rst[(rst < 0)] = 0
        
        # You can then write out a new file
        meta = src.meta
        print(meta)
        #meta['width'], meta['height'] = rst.shape[1],rst.shape[0]
        meta['width'], meta['height'] = rst.shape[0],rst.shape[1]

        meta['transform'] = rio.windows.transform(my_window, src.transform)
        print(meta)

        with rio.open(outfile, 'w', **meta) as dst:
            dst.write(rst, 1)

        return rst

In [4]:
import matplotlib.pyplot as pyplot
from rasterio.plot import show

def my_plot(array):
    cmaps = ['Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r']
    axs=()
    fig, axs = pyplot.subplots(1,8, figsize=(21,21))
    for i in range(0,8):
        show(array, ax=axs[i], cmap=cmaps[i], title=cmaps[i])
    pyplot.show()

In [5]:
%%time
geoj = 'great_lakes_edit.geojson'

for i in range(1,366,10):
    tif = '/vsis3/dev-et-data/compressed/NDVI_filled/2001/2001{0:03}.250_m_NDVI.tif'.format(i)
    outf = 'data/gl{0:03}.tif'.format(i)
    print(tif)
    print(outf)
    array = clip_by_geojson(tif,outf,geoj)
    print(array.shape)
    my_plot(array)

/vsis3/dev-et-data/compressed/NDVI_filled/2001/2001001.250_m_NDVI.tif
data/gl001.tif
                                            geometry
0  POLYGON ((-90.00000 49.90000, -80.00000 49.900...
-90.0 -80.0 49.9 40.0


RasterioIOError: Connection timed out after 1001 milliseconds

In [6]:
! ls data

ls: cannot access 'data': No such file or directory


In [7]:
%%bash
for i in `ls data`; do gdalinfo ./data/$i | grep 'Size is'; done

ls: cannot access 'data': No such file or directory


In [8]:
%%bash
file=
AWS_REQUEST_PAYER=requester GDAL_DISABLE_READDIR_ON_OPEN=YES CPL_VSIL_CURL_ALLOWED_EXTENSIONS=.tiff \
rio info  /vsis3/dev-et-data/compressed/NDVI_filled/2001/2001001.250_m_NDVI.tif

Traceback (most recent call last):
  File "rasterio/_base.pyx", line 216, in rasterio._base.DatasetBase.__init__
  File "rasterio/_shim.pyx", line 78, in rasterio._shim.open_dataset
  File "rasterio/_err.pyx", line 205, in rasterio._err.exc_wrap_pointer
rasterio._err.CPLE_OpenFailedError: '/vsis3/dev-et-data/compressed/NDVI_filled/2001/2001001.250_m_NDVI.tif' does not exist in the file system, and is not recognized as a supported dataset name.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/pangeo/bin/rio", line 11, in <module>
    sys.exit(main_group())
  File "/srv/conda/envs/pangeo/lib/python3.7/site-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/srv/conda/envs/pangeo/lib/python3.7/site-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/srv/conda/envs/pangeo/lib/python3.7/site-packages/click/core.py", line 1259, in invoke
    retu

CalledProcessError: Command 'b'file=\nAWS_REQUEST_PAYER=requester GDAL_DISABLE_READDIR_ON_OPEN=YES CPL_VSIL_CURL_ALLOWED_EXTENSIONS=.tiff \\\nrio info  /vsis3/dev-et-data/compressed/NDVI_filled/2001/2001001.250_m_NDVI.tif\n'' returned non-zero exit status 1.

In [12]:
%env AWS_REQUEST_PAYER=requester
!(AWS_REQUEST_PAYER=requester; rio info  /vsis3/dev-et-data/compressed/NDVI_filled/2001/2001001.250_m_NDVI.tif)

env: AWS_REQUEST_PAYER=requester
Traceback (most recent call last):
  File "rasterio/_base.pyx", line 216, in rasterio._base.DatasetBase.__init__
  File "rasterio/_shim.pyx", line 78, in rasterio._shim.open_dataset
  File "rasterio/_err.pyx", line 205, in rasterio._err.exc_wrap_pointer
rasterio._err.CPLE_AppDefinedError: Connection timed out after 1001 milliseconds

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/pangeo/bin/rio", line 11, in <module>
    sys.exit(main_group())
  File "/srv/conda/envs/pangeo/lib/python3.7/site-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/srv/conda/envs/pangeo/lib/python3.7/site-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/srv/conda/envs/pangeo/lib/python3.7/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/srv/conda/envs/pang

In [14]:
!rio info /vsis3/dev-et-data/compressed/NDVI_filled/2001/2001001.250_m_NDVI.tif

Usage: rio info [OPTIONS] INPUT
Try 'rio info --help' for help.

Error: no such option: --requester-pays
